In [1]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

In [4]:
stopwords = Stopwords()
tokenizer = Kiwi()

In [26]:
def tokenize(text):
    tokens = tokenizer.tokenize(text, stopwords=stopwords)
    return [ (pos.form, pos.tag) for pos in tokens]

In [29]:
from rank_bm25 import BM25Okapi
import pickle
import pandas as pd
from tqdm import tqdm

df = pd.read_excel('../resources/comfort_datasets.xlsx')

corpus = df['user'].tolist()
tokenized_corpus = [tokenize(doc) for doc in tqdm(corpus)]
bm25 = BM25Okapi(tokenized_corpus)

# To save bm25 object
with open('bm25result', 'wb') as bm25result_file:
    pickle.dump(bm25, bm25result_file)

100%|███████████████████████████████████| 34477/34477 [00:16<00:00, 2052.29it/s]


In [39]:
import numpy as np

In [40]:
# to read bm25 object
with open('bm25result', 'rb') as bm25result_file:
    bm25result = pickle.load(bm25result_file)

In [71]:
def bm25_search(query, n=5):
    tokenized_query = tokenize(query)
    topk = bm25result.get_top_n(tokenized_query, corpus, n=n)
    return topk


def bm25_chatbot_response(query, n=5):
    tokenized_query = tokenize(query)
    scores = bm25result.get_scores(tokenized_query)
    top_5_indices = np.argsort(scores)[-5:]
    res = df.loc[top_5_indices[::-1]]
    return res

In [77]:
bm25_chatbot_response("짝사랑 포기하는 방법").index.to_list()

[30948, 30935, 30949, 30947, 29123]